# Radiative kernel feedback analysis for Ford et al. (2025)

This notebook performs the radiative kernel feedback analysis, which is used in Figs. 5 and 6. The code in this notebook is based on the demo code here:

Angeline G Pendergrass. (2019). apendergrass/cam5-kernels: Up to date codebase as of August 2019 (August-2019). Zenodo. https://doi.org/10.5281/zenodo.3359041.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.lines as mlines
import xarray as xr
import xesmf as xe
from cartopy import crs as ccrs
import cartopy.feature as cf
import matplotlib.colors as colors
from climlab.utils.thermo import qsat
import glob, os
import s3fs
import fsspec

Define various functions:

In [2]:
def global_average(data, lon=True):
    weights = np.cos(np.deg2rad(data.lat))
    data_weighted = data.weighted(weights)
    if lon == True: 
        return data_weighted.mean(dim=['lat', 'lon'], skipna=True)
    else: return data_weighted.mean(dim='lat', skipna=True)

In [3]:
def regrid(ds_in, regrid_to, method='bilinear'):
    regridder = xe.Regridder(ds_in, regrid_to, method=method, periodic=True, ignore_degenerate=True)
    ds_out = regridder(ds_in)
    return ds_out

In [4]:
def tropo_mask(ds):
    return ds.where(ds.lev > ((200/90)*abs(ds.lat) + 100))

In [5]:
def cesm_kernel_analysis(ctrl, expr, kts, kt, kq, ka, ghgfile, aerfile, monthly=True, return_opts='gm'):
        
    # Temperature
    dta = regrid(expr.T - ctrl.T, kt)
    dts = regrid(expr.TS - ctrl.TS, kts)
    
    if monthly is True:
        dts_gm = global_average(dts.mean(dim='month'))
    else: dts_gm = global_average(dts)
    
    ts_kernel = kts.FLNT
    ts_kernel_cs = kts.FLNTC
    dLW_ts = ts_kernel * dts  
    dLW_ts_cs = ts_kernel_cs * dts
    
    ta_kernel = kt.FLNT
    ta_kernel_cs = kt.FLNTC
    dLW_ta = (ta_kernel * tropo_mask(dta)).sum(dim='lev')
    dLW_ta_cs = (ta_kernel_cs * tropo_mask(dta)).sum(dim='lev')
    
    if monthly is True:
        dLW_ts = dLW_ts.mean(dim='month')
        dLW_ts_cs = dLW_ts_cs.mean(dim='month')
        dLW_ta = dLW_ta.mean(dim='month')
        dLW_ta_cs = dLW_ta_cs.mean(dim='month')
    
    dLW_t_map = -dLW_ta - dLW_ts
    dLW_t_gm = global_average(dLW_t_map)
    
    t_feedback_map = dLW_t_map/dts_gm
    t_feedback = dLW_t_gm/dts_gm
    
    # Planck
    dts_3d = dts.expand_dims(dim={'lev': dta.lev})
    dt_planck = tropo_mask(dts_3d)
    dLW_planck = (ta_kernel * dt_planck).sum(dim='lev')
    
    if monthly is True:
        dLW_planck = dLW_planck.mean(dim='month')
    
    dLW_planck_map = -dLW_planck - dLW_ts
    dLW_planck_gm = global_average(dLW_planck_map)
    
    planck_feedback_map = dLW_planck_map/dts_gm
    planck_feedback = dLW_planck_gm/dts_gm

    # LR
    dt_lapserate = tropo_mask(dta - dt_planck)
    dLW_lapserate = (ta_kernel * dt_lapserate).sum(dim='lev')
    
    if monthly is True:
        dLW_lapserate = dLW_lapserate.mean(dim='month')
        
    dLW_lapserate_map = -dLW_lapserate
    dLW_lapserate_gm = global_average(dLW_lapserate_map)
    
    lapserate_feedback_map = dLW_lapserate_map/dts_gm
    lapserate_feedback = dLW_lapserate_gm/dts_gm

    # Albedo
    alb_ctrl = 1 - (ctrl.FSNS/ctrl.FSDS)
    alb_expr = 1 - (expr.FSNS/expr.FSDS)
    dalb = (alb_expr - alb_ctrl)*100
    dSW_alb = ka.FSNT * regrid(dalb, ka)
    dSW_alb_cs = ka.FSNTC * regrid(dalb, ka)
    
    if monthly is True:
        dSW_alb = dSW_alb.mean(dim='month')
        dSW_alb_cs = dSW_alb_cs.mean(dim='month')
    
    a_feedback_map = dSW_alb/dts_gm
    a_feedback = global_average(dSW_alb)/dts_gm

    # WV
    dlogq = np.log(regrid(expr.Q, kq)) - np.log(regrid(ctrl.Q, kq))
    small = 0.01
    dlogqsatdt = (np.log(qsat(regrid(ctrl.T, kt)+small, ctrl.lev)) - np.log(qsat(regrid(ctrl.T, kt)-small, ctrl.lev))) / (2*small)
    dteq_log = tropo_mask(dlogq/dlogqsatdt)

    q_LW_kernel = kq.FLNT
    q_SW_kernel = kq.FSNT
    q_LW_kernel_cs = kq.FLNTC
    q_SW_kernel_cs = kq.FSNTC
    
    dLW_q = (q_LW_kernel * dteq_log).sum(dim='lev')
    dSW_q = (q_SW_kernel * dteq_log).sum(dim='lev')
    dLW_q_cs = (q_LW_kernel_cs * dteq_log).sum(dim='lev')
    dSW_q_cs = (q_SW_kernel_cs * dteq_log).sum(dim='lev')
    
    if monthly is True:
        dLW_q = dLW_q.mean(dim='month')
        dSW_q = dSW_q.mean(dim='month')
        dLW_q_cs = dLW_q_cs.mean(dim='month')
        dSW_q_cs = dSW_q_cs.mean(dim='month')
    
    q_feedback_map = (-dLW_q+dSW_q)/dts_gm
    q_feedback = global_average(-dLW_q+dSW_q)/dts_gm
    
    lw_q_fb_map = -dLW_q/dts_gm
    lw_q_fb = global_average(-dLW_q)/dts_gm

    sw_q_fb_map = dSW_q/dts_gm
    sw_q_fb = global_average(dSW_q)/dts_gm
    
    # Cloud
    d_sw = expr.FSNT - ctrl.FSNT
    d_sw_cs = expr.FSNTC - ctrl.FSNTC
    d_lw = expr.FLNT - ctrl.FLNT
    d_lw_cs = expr.FLNTC - ctrl.FLNTC
    d_cre_sw = d_sw_cs - d_sw
    d_cre_lw = d_lw_cs - d_lw

    ghgfile_sw = ghgfile.FSNT
    ghgfile_sw_cs = ghgfile.FSNTC
    ghgfile_lw = ghgfile.FLNT
    ghgfile_lw_cs = ghgfile.FLNTC
    ghg_sw = ghgfile_sw_cs - ghgfile_sw
    ghg_lw = ghgfile_lw_cs - ghgfile_lw
    aerfile_sw = aerfile.FSNT
    aerfile_sw_cs = aerfile.FSNTC
    aerfile_lw = aerfile.FLNT
    aerfile_lw_cs = aerfile.FLNTC
    aer_sw = aerfile_sw_cs - aerfile_sw
    aer_lw = aerfile_lw_cs - aerfile_lw
    cloud_masking_of_forcing_sw = aer_sw + ghg_sw
    cloud_masking_of_forcing_lw = aer_lw + ghg_lw

    dLW_cloud = -regrid(d_cre_lw, kts) + cloud_masking_of_forcing_lw + (dLW_q_cs-dLW_q) + (dLW_ta_cs-dLW_ta) + (dLW_ts_cs-dLW_ts)
    dSW_cloud = -regrid(d_cre_sw, kts) + cloud_masking_of_forcing_sw + (dSW_q_cs-dSW_q) + (dSW_alb_cs-dSW_alb)
    
    if monthly is True:
        dLW_cloud = dLW_cloud.mean(dim='month')
        dSW_cloud = dSW_cloud.mean(dim='month')
    
    lw_c_fb_map = -dLW_cloud/dts_gm
    lw_c_fb = global_average(-dLW_cloud)/dts_gm

    sw_c_fb_map = dSW_cloud/dts_gm
    sw_c_fb = global_average(dSW_cloud)/dts_gm
    
    # Total
    total_feedback = planck_feedback + lapserate_feedback + q_feedback + a_feedback + sw_c_fb + lw_c_fb
    total_feedback_map = planck_feedback_map + lapserate_feedback_map + q_feedback_map + a_feedback_map + sw_c_fb_map + lw_c_fb_map

    # Dictionaries to return
    feedbacks = {}
    feedbacks['Total'] = total_feedback.load()
    
    feedbacks['Planck'] = planck_feedback.load()
    feedbacks['LR'] = lapserate_feedback.load()
    feedbacks['Net_T'] = feedbacks['Planck'] + feedbacks['LR']
    
    feedbacks['SW_q'] = sw_q_fb.load()
    feedbacks['LW_q'] = lw_q_fb.load()
    feedbacks['Net_q'] = feedbacks['SW_q'] + feedbacks['LW_q']
    
    feedbacks['SW_cloud'] = sw_c_fb.load()
    feedbacks['LW_cloud'] = lw_c_fb.load()
    feedbacks['Net_cloud'] = feedbacks['SW_cloud'] + feedbacks['LW_cloud']

    feedbacks['Albedo'] = a_feedback.load()

    feedbacks_map = {}
    feedbacks_map['Total'] = total_feedback_map.load()
    
    feedbacks_map['Planck'] = planck_feedback_map.load()
    feedbacks_map['LR'] = lapserate_feedback_map.load()
    feedbacks_map['Net_T'] = feedbacks_map['Planck'] + feedbacks_map['LR']

    feedbacks_map['SW_q'] = sw_q_fb_map.load()
    feedbacks_map['LW_q'] = lw_q_fb_map.load()
    feedbacks_map['Net_q'] = feedbacks_map['SW_q'] + feedbacks_map['LW_q']
        
    feedbacks_map['SW_cloud'] = sw_c_fb_map.load()
    feedbacks_map['LW_cloud'] = lw_c_fb_map.load()
    feedbacks_map['Net_cloud'] = feedbacks_map['SW_cloud'] + feedbacks_map['LW_cloud']

    feedbacks_map['Albedo'] = a_feedback_map.load()
    
    if return_opts == 'gm':
        return feedbacks
    if return_opts == 'map':
        return feedbacks_map
    if return_opts == 'both':
        return {'gm': feedbacks, 'map': feedbacks_map}

In [6]:
def fix_dim(ds):
    ds['time'] = np.arange(1, 13)
    return ds.rename_dims({'time': 'month'})

## Load kernels and data

If you are running this notebook locally, the kernel and forcing data should be downloaded from https://zenodo.org/records/997902, with the filepaths below changed accordingly (the filenames here are the same as the originals):

In [7]:
kq = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/kernels/q.kernel.nc'))
kt = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/kernels/t.kernel.nc'))
ka = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/kernels/alb.kernel.nc'))
kts = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/kernels/ts.kernel.nc'))
ghgfile = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/forcing/ghg.forcing.nc'))
aerfile = fix_dim(xr.open_dataset('/roselab_rit/rford/OHUTCR-data/cam5-kernels/forcing/aerosol.forcing.nc'))

kq_ann = kq.mean(dim='month')
kt_ann = kt.mean(dim='month')
ka_ann = ka.mean(dim='month')
kts_ann = kts.mean(dim='month')
ghgfile_ann = ghgfile.mean(dim='month')
aerfile_ann = aerfile.mean(dim='month')

In [8]:
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/OHUTCR-data'
fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))
pattern = f's3://{path}/*.nc'
files = sorted(fs.glob(pattern))
fileset = [fs.open(file) for file in files]
fileset

[<File-like object S3FileSystem, pythia/OHUTCR-data/CESM1-CAM5_CO2.cam.h0.mean.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/CESM1-CAM5_ctrl.cam.h0.mean.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/OHUTCR-feedbacks.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/cmip5em-tcr-mean-output.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/cold4-model-tcr-mean-output.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/pop_frc.b.e11.B1850C5CN.f19_g16.130429.nc>,
 <File-like object S3FileSystem, pythia/OHUTCR-data/warm13-model-tcr-mean-output.nc>]

In [9]:
expr = xr.open_dataset(fileset[6]).drop_sel(model='GFDL-ESM2G')

In [10]:
expr_em = xr.open_dataset(fileset[3])

In [11]:
expr_2x = xr.open_dataset(fileset[0]).isel(time=0)

In [12]:
ctrl = xr.open_dataset(fileset[1]).isel(time=0)

## Do the analysis

In [13]:
feedbacks = cesm_kernel_analysis(ctrl, expr, kts_ann, kt_ann, kq_ann, ka_ann, ghgfile_ann, aerfile_ann, monthly=False, return_opts='both')

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.app

In [14]:
feedbacks_2x = cesm_kernel_analysis(ctrl, expr_2x, kts_ann, kt_ann, kq_ann, ka_ann, ghgfile_ann, aerfile_ann, monthly=False, return_opts='both')

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.app

In [15]:
feedbacks_em = cesm_kernel_analysis(ctrl, expr_em, kts_ann, kt_ann, kq_ann, ka_ann, ghgfile_ann, aerfile_ann, monthly=False, return_opts='both')

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.app

Merge all data into one Dataset:

In [16]:
feedback_maps_ds = feedbacks['map']
feedback_2x_maps = feedbacks_2x['map']
feedback_em_maps = feedbacks_em['map']

In [17]:
for key in feedback_maps_ds.keys():
    feedback_maps_ds[key] = feedback_maps_ds[key].to_dataset(name=key)

In [18]:
for key in feedback_2x_maps.keys():
    feedback_2x_maps[key] = feedback_2x_maps[key].to_dataset(name=key)

In [19]:
for key in feedback_em_maps.keys():
    feedback_em_maps[key] = feedback_em_maps[key].to_dataset(name=key)

In [20]:
fb_merged = xr.merge(feedback_maps_ds.values())

In [21]:
fb_2x_merged = xr.merge(feedback_2x_maps.values())

In [22]:
fb_em_merged = xr.merge(feedback_em_maps.values())

In [23]:
fb_2x_merged_exp = fb_2x_merged.expand_dims('model')

In [24]:
fb_2x_merged_ = fb_2x_merged.assign_coords({'model': '2xCO2-only'}).expand_dims('model')
fb_em_merged_ = fb_em_merged.assign_coords({'model': 'CMIP5-EM'}).expand_dims('model')

In [25]:
fb_all_merged = xr.merge([fb_merged, fb_2x_merged_, fb_em_merged_])

Uncomment and edit the below line to save the file:

In [ ]:
# fb_all_merged.to_netcdf('/roselab_rit/rford/OHUTCR-data/OHUTCR-feedbacks.nc')